# COVID-19 TF-IDF and BM25 Based Information Retrieval System

Dalam notebook ini, kita membangun sistem pengambilan informasi sederhana menggunakan TF-IDF untuk menghasilkan array indeks terbalik, dan kemudian menggunakan BM25 untuk melakukan pencarian dan pengindeksan artikel.

Langkah-langkah:
1. Ekstraksi: Mengambil data dari dataset.
2. Organisasi: Membuat dictionary untuk menyimpan informasi relevan dan menghasilkan indeks terbalik berdasarkan kata kunci dari setiap dokumen.
3. Pengambilan: Menjalankan algoritma BM25 untuk menghasilkan daftar hasil yang telah diurutkan.

## Langkah-Langkah Utama
1. Ekstraksi Data
Mengunduh file JSON dari koleksi dataset COVID-19, mengekstraknya, dan menyimpannya dalam folder data.

2. Pra-pemrosesan Teks
Melakukan:
- Penghapusan stopword, tanda baca, angka, dan karakter khusus.
- Konversi teks menjadi huruf kecil.
- Stemming dan lemmatization.
3. Ekstraksi Informasi
Mengambil paper_id, judul, dan abstrak dari setiap artikel, lalu menyimpannya dalam DataFrame.

4. Organisasi Data
- Menghitung skor TF-IDF untuk setiap kata dalam abstrak.
- Membuat indeks terbalik untuk mempercepat pencarian.
5. Pengambilan Informasi
- Menggunakan BM25 untuk menghitung relevansi artikel terhadap query.
- Mengurutkan artikel berdasarkan skor relevansi.

## Contoh Query

In [ ]:
q = ['asal-usul coronavirus', 'respons coronavirus terhadap perubahan cuaca', 'imunitas coronavirus']

Hasilnya adalah daftar artikel yang paling relevan untuk setiap query, diurutkan berdasarkan skor BM25.

## Library yang Digunakan  
- Ekstraksi Data: os, urllib.request, tarfile, json, pandas, numpy, tqdm
- Pra-pemrosesan Teks: re, nltk (stopwords, stemming, lemmatization)
- Ekstraksi Kata Kunci: sklearn.feature_extraction.text, scipy.sparse
- BM25: rank_bm25

In [24]:
#libraries for getting data and extracting
import os
import urllib.request
import tarfile
import json
import pandas as pd
import numpy as np
from tqdm import tqdm


#libraries for text preprocessing
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer

#libraries for keyword extraction with tf-idf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import coo_matrix

#libraries for reading and writing files
import pickle

#libraries for BM25
# !pip install rank_bm25
from rank_bm25 import BM25Okapi

[nltk_data] Downloading package stopwords to C:\Users\Tuhu
[nltk_data]     Pangestu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Tuhu
[nltk_data]     Pangestu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Ekstraksi Data

Mengambil data

Mengunduh file-file JSON dari koleksi dan menempatkannya ke dalam satu folder bernama data. Di dalam folder data, dibuat subfolder berdasarkan asal masing-masing artikel. Kita akan menggunakan file-file JSON yang disimpan dalam folder data ini pada langkah ekstraksi untuk membuat dataset kita.

In [25]:
def getData():
    urls = ['https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-03-27/comm_use_subset.tar.gz', 'https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-03-27/noncomm_use_subset.tar.gz', 'https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-03-27/custom_license.tar.gz', 'https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-03-27/biorxiv_medrxiv.tar.gz']

    # Create data directory
    try:
        os.mkdir('./data')
        print('Directory created')
    except FileExistsError:
        print('Directory already exists')

    #Download all files
    for i in range(len(urls)):
        urllib.request.urlretrieve(urls[i], './data/file'+str(i)+'.tar.gz')
        print('Downloaded file '+str(i+1)+'/'+str(len(urls)))
        tar = tarfile.open('./data/file'+str(i)+'.tar.gz')
        tar.extractall('./data')
        tar.close()
        print('Extracted file '+str(i+1)+'/'+str(len(urls)))
        os.remove('./data/file'+str(i)+'.tar.gz')

### Pra-pemrosesan
Input: Teks  
Output: Teks yang telah dipra-proses – menghapus stopword, tanda baca, mengubah semua huruf menjadi huruf kecil, menghapus angka/karakter khusus, melakukan stemming, dan lemmatization.  

Langkah ini diperlukan untuk tahap ekstraksi karena kita akan melakukan pra-pemrosesan terhadap semua teks yang diekstrak (judul dan abstrak).

In [26]:
def preprocess(text):
    #define stopwords
    stop_words = list(stopwords.words('english'))
    #Remove punctuations
    text = re.sub('[^a-zA-Z]', ' ', text)
    #Convert to lowercase
    text = text.lower()
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    ##Convert to list from string
    text = text.split()
    ##Stemming
    ps=PorterStemmer()
    text = [ps.stem(word) for word in text if not word in stop_words]
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  stop_words] 
    text = " ".join(text) 
    
    return text

### Ekstraksi

Input: Tidak ada  
Output: Sebuah dataframe yang berisi semua informasi yang telah diekstrak  

Untuk setiap artikel, kita mengumpulkan:
- paper_id
- Judul
- Abstrak

Kita menyimpan nilai-nilai ini ke dalam sebuah dataframe menggunakan pustaka pandas, lalu menyimpannya ke dalam file pickle.

In [27]:
def extract():
    #create our collection locally in the data folder
    
    #creating our initial datastructure
    x = {'paper_id':[], 'title':[], 'abstract': []}
    
    #Iterate through all files in the data directory
    for subdir, dirs, files in os.walk('./data'):
        for file in tqdm(files):
            with open(os.path.join(subdir, file)) as f:
                data = json.load(f)
                
               #Append paper ID to list
                x['paper_id'].append(data['paper_id'])
               #Append article title to list & preprocess the text
                x['title'].append((data['metadata']['title']))
                
                #Append abstract text content values only to abstract list & preprocess the text
                abstract = ""
                for paragraph in data['abstract']:
                    abstract += paragraph['text']
                    abstract += '\n'
                #if json file no abstract in file, set the body text as the abstract (happens rarely, but often enough that this edge case matters)
                if abstract == "": 
                    for paragraph in data['body_text']:
                        abstract += paragraph['text']
                        abstract += '\n'
                x['abstract'].append(preprocess(abstract))
                
    #Create Pandas dataframe & write to pickle file
    df = pd.DataFrame.from_dict(x, orient='index')
    df = df.transpose()
    pickle.dump( df, open( "full_data_processed_FINAL.p", "wb" ) )
    return df

**Organize**

Urutkan Matriks Koordinat  
Input: representasi koordinat tf_idf  
Output: item tf_idf yang diurutkan dalam urutan menurun — sehingga item dengan skor tertinggi berada di atas!  

Fungsi untuk mengurutkan tf_idf dalam urutan menurun.

In [28]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

Mengambil Top N Kata dengan Skor TF‑IDF Tertinggi
Input:
1. feature_names = daftar kosakata
2. sorted_items = vektor TF‑IDF yang sudah diurutkan menurun
3. topN = jumlah kata kunci yang ingin Anda ekstrak dari teks

Dictionary berisi topN kata dengan skor TF‑IDF tertinggi dalam teks (sebagai key) beserta skor TF‑IDF masing‑masing (sebagai value)  
Fungsi ini mengembalikan nama kata kunci dan skor TF‑IDF mereka untuk topN item teratas.

In [29]:
def extract_topn_from_vector(feature_names, sorted_items, topN):
    #use only topn items from vector
    sorted_items = sorted_items[:topN]
 
    score_vals = []
    feature_vals = []
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    return results

Mengambil Kata Kunci dari Abstrak  

Input:
1. entry = baris dalam DataFrame artikel, yang mewakili satu artikel
2. cv = objek CountVectorizer dari sklearn.feature_extraction.text
3. X = matriks hasil CountVectorizer yang sudah di-fit pada korpus
4. tfidf_transformer = objek yang menyimpan data TF‑IDF — sudah di-fit pada korpus
5. feature_names = daftar kosakata
6. topN = jumlah kata kunci yang ingin diambil dari abstrak

Output:
TopN kata kunci teratas dari abstrak  
  
Fungsi ini mengekstrak dan mengembalikan topN kata kunci dengan skor TF‑IDF tertinggi dari abstrak.

In [30]:
def getAbstractKeywords(entry, cv, X, tfidf_transformer, feature_names, topN):
    abstract = entry['abstract']
    
    #first check that abstract is full
    if type(abstract) == float:
        return []
 
    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([abstract])) 
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    #extract only the topN # items
    keywords_dict=extract_topn_from_vector(feature_names,sorted_items,topN)
    #just want words themselves, so only need keys of the dictionary
    keywords = list(keywords_dict.keys()) 
     
    return keywords

Mengambil Kata Kunci dari Judul  
**Input:**  
entry = baris dalam DataFrame artikel, yang mewakili satu artikel  
**Output:**    
Daftar semua kata yang terdapat dalam judul  
  
Diandaikan bahwa jika sebuah kata terdapat dalam judul artikel, maka kata tersebut penting untuk artikel dan diperlakukan sebagai kata kunci. Oleh karena itu, metode ini hanya mengekstrak semua kata dari judul yang telah diproses sebelumnya.

In [31]:
def getTitleKeywords(entry):
    title = entry['title']  
    title = preprocess(title)
    #first check that the title of that entry is full
    if type(title) == float:
        return []
    
    keywords_title = title.split(' ')
    return keywords_title

Mengambil keyword final  
Input:  

1. entry = baris dalam dataframe artikel, yang merepresentasikan satu artikel
2. cv = CountVectorizer dari sklearn.feature_extraction.text
3. X = vektor yang merepresentasikan hasil fit CountVectorizer terhadap korpus
4. tfidf_transformer = objek yang menyimpan data tf-idf — juga sudah difit ke korpus
5. feature_names = daftar kosakata
6. topN = jumlah keyword yang ingin diekstrak dari abstrak

Output:  
Daftar semua keyword untuk sebuah artikel — diekstrak dari judul dan abstrak!

Fungsi ini memanggil getTitleKeywords() dan getAbstractKeywords(), lalu menggabungkan kedua daftar tersebut menjadi satu daftar keyword final.

In [32]:
def getFinalKeywords(entry, cv, X, tfidf_trans, feature_names, topN):
    #get keywords from abstract and title
    fromAbstract = getAbstractKeywords(entry, cv, X, tfidf_trans, feature_names, topN)
    fromTitle = getTitleKeywords(entry)
    #concatenate two lists
    finalKeywords = fromAbstract + fromTitle
    #convert to set and then back to list to ensure there are no duplicates in list
    final_no_duplicates = list(set(finalKeywords))
    return final_no_duplicates

Mengambil korpus
Input:  
Sebuah dataframe yang berisi:  
1. paper_id
2. abstract
3. article (untuk setiap artikel. Semua teks sudah diproses.)
  
Output:  
Sebuah daftar (list) yang berisi seluruh abstrak dalam dataframe artikel.  
  
Langkah ini digunakan untuk membuat corpus, yang merupakan input penting dalam proses tf-idf kita.

In [33]:
def getCorpus(articlesDf):
    #creating a new dataframe, abstractDf, of just the abstracts, so that we don't modify the original dataframe, articlesDf
    abstractDf = pd.DataFrame(columns = ['abstract'])
    #filling abstractDf with the abstract column from articlesDf
    abstractDf['abstract'] = articlesDf['abstract']
    #converting column of dataframe to a list
    corpus = abstractDf['abstract'].to_list()
    return corpus


**Menambahkan keyword**  
Input:  
1. df = DataFrame yang berisi:  
    - paper_id
    - title
    - abstract (untuk setiap artikel. Semua teks sudah diproses.)

2. top N = jumlah keyword yang ingin diekstrak dari abstrak
3. makeFile = nilai boolean, menentukan apakah fungsi ini akan membuat file pickle dari dataframe hasil
4. fileName = nama file pickle yang diinginkan oleh pengguna
  
Output:  
Sebuah DataFrame pandas yang berisi:  
1. paper_id
2. title
3. abstract
4. keyword yang dikaitkan dengan setiap artikel

In [34]:
def addKeywords(df, topN, makeFile, fileName):
    #defining stopwords
    stop_words = list(stopwords.words("english"))

    #creating following variables that are needed for keyword extract from abstract, using tf-idf methodology,
    #all input in getFinalKewords method
    corpus = getCorpus(df)
    cv=CountVectorizer(max_df=0.8,stop_words=stop_words, max_features=1000, ngram_range=(1,1))    
    X=cv.fit_transform(corpus)
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    tfidf_transformer.fit(X)
    feature_names=cv.get_feature_names_out()
    
    #adding keywords article to dataframe
    df = df.reindex(columns = ['paper_id', 'title', 'abstract','keywords'])                
    #getting keywords for each entry in article dataframe -- using apply to be more efficient
    df['keywords'] = df.apply(lambda row: getFinalKeywords(row, cv, X, tfidf_transformer, feature_names, topN), axis=1)

    #make pickle file depending on user input
    if makeFile == True:
        pickle.dump( df, open( fileName, "wb" ) )
    return df  

Membuat inverted indices  
Input:  
Sebuah DataFrame pandas yang berisi:  
1. paper_id
2. title
3. abstract
4. keywords yang dikaitkan dengan setiap artikel
  
Output:  
Sebuah dictionary dari inverted indices  
- key = kata yang merupakan keyword
- value = semua paper_id dari artikel yang memiliki kata tersebut sebagai keyword
  
Fungsi ini membuat dictionary inverted index, yang nantinya akan digunakan saat menentukan subset artikel untuk menjalankan algoritma pencarian atau peringkat. Output-nya penting berupa dictionary karena memiliki waktu pencarian yang konstan (constant look-up time).

In [35]:
def createInvertedIndices(df):
    numEntries = df.shape[0]
    invertInd = {}
    
    for i in range (numEntries):
        entry = df.iloc[i]
        paper_id = entry['paper_id']    
        keywords = entry['keywords']
        for k in keywords:
            if k not in invertInd:
                invertInd[k] = []
                invertInd[k].append(paper_id)
            else:
                invertInd[k].append(paper_id)
    return invertInd

organize step

In [36]:
def organize():
    df_without_keywords = pickle.load(open("full_data_processed_FINAL.p", "rb"))
    df_with_keywords = addKeywords(df_without_keywords, 10, False, "full_data_withKeywords_FINAL.p")
    invertedIndices = createInvertedIndices(df_with_keywords)
    pickle.dump( invertedIndices, open( "invertedIndices_FINAL.p", "wb" ) )

**Data setelah diproses**

In [37]:
# getData()
# extract()

df_without_keywords = pickle.load(open("full_data_processed_FINAL.p", "rb"))
df_with_keywords = addKeywords(df_without_keywords, 10, False, "full_data_withKeywords_FINAL.p")
display(df_without_keywords)
display(df_with_keywords)

,paper_id,title,abstract
0,0015023cc06b5362d332b3baf348d11567ca2fbb,The RNA pseudoknots in foot-and-mouth disease ...,word count text word count author funder right...
1,00340eea543336d54adda18236424de6a5e91c9d,Analysis Title: Regaining perspective on SARS-...,past three month new coronaviru sar cov epidem...
2,004f0f8bb66cf446678dc13cf2701feec4f36d76,Healthcare-resource-adjusted vulnerabilities t...,ncov epidem spread across china countri februa...
3,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,"Real-time, MinION-based, amplicon sequencing f...",infecti bronchiti ib caus signific econom loss...
4,0139ea4ca580af99b602c6435368e7fdbefacb03,A Combined Evidence Approach to Prioritize Nip...,nipah viru niv came limelight recent due outbr...
...,...,...,...
33370,ff5a79ed22ea416e6d89caad1cf0d83dbc741a4b,Understanding Human Coronavirus HCoV-NL63,even though coronaviru infect human normal ass...
33371,ff6d57f2aad99be129432058665b361dc18747e8,Brief Definitive Report MACROPHAGES GENETICALL...,extens evid cultur macrophag genet resist susc...
33372,ff83907653a4c4500e8c509ca28169e924742b40,Identification of a Subdomain of CENPB That Is...,combin vivo vitro approach investig function c...
33373,ffe718db1820f27bf274e3fc519ab78e450de288,Replication enhancer elements within the open ...,provid experiment evid replic enhanc element r...


,paper_id,title,abstract,keywords
0,0015023cc06b5362d332b3baf348d11567ca2fbb,The RNA pseudoknots in foot-and-mouth disease ...,word count text word count author funder right...,"[genom, count, product, replic, nucleotid, len..."
1,00340eea543336d54adda18236424de6a5e91c9d,Analysis Title: Regaining perspective on SARS-...,past three month new coronaviru sar cov epidem...,"[trace, base, perspect, molecular, light, anal..."
2,004f0f8bb66cf446678dc13cf2701feec4f36d76,Healthcare-resource-adjusted vulnerabilities t...,ncov epidem spread across china countri februa...,"[ncov, healthcar, local, health, public, numbe..."
3,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,"Real-time, MinION-based, amplicon sequencing f...",infecti bronchiti ib caus signific econom loss...,"[time, serotyp, upper, use, lineag, viru, rt, ..."
4,0139ea4ca580af99b602c6435368e7fdbefacb03,A Combined Evidence Approach to Prioritize Nip...,nipah viru niv came limelight recent due outbr...,"[pain, base, diver, inhibitor, drug, combin, r..."
...,...,...,...,...
33370,ff5a79ed22ea416e6d89caad1cf0d83dbc741a4b,Understanding Human Coronavirus HCoV-NL63,even though coronaviru infect human normal ass...,"[child, mainli, infect, hcov, infant, respirat..."
33371,ff6d57f2aad99be129432058665b361dc18747e8,Brief Definitive Report MACROPHAGES GENETICALL...,extens evid cultur macrophag genet resist susc...,"[macrophag, vitro, viru, genet, report, labora..."
33372,ff83907653a4c4500e8c509ca28169e924742b40,Identification of a Subdomain of CENPB That Is...,combin vivo vitro approach investig function c...,"[subdomain, suffici, local, dna, protein, repe..."
33373,ffe718db1820f27bf274e3fc519ab78e450de288,Replication enhancer elements within the open ...,provid experiment evid replic enhanc element r...,"[mutant, replic, read, frame, enhanc, infect, ..."


RETRIEVE

mendapatkan subset artikel kita
Input: query
Output: daftar artikel potensial yang mungkin relevan, karena memiliki beberapa istilah query sebagai kata kunci mereka

Langkah ini dilakukan agar kita tidak perlu menjalankan algoritma peringkat, BM25, pada semua ~30.000 artikel. Tujuannya adalah untuk mengidentifikasi subset ini sehingga proses peringkat (dan pengambilan) menjadi lebih efisien!

In [38]:
def getPotentialArticleSubset(query):
    #load in inverted indices
    invertedIndices = pickle.load(open("invertedIndices_FINAL.p", "rb"))
    
    #preprocess query and split into individual terms
    query = preprocess(query)
    queryTerms = query.split(' ')
    
    potentialArticles = []
    #concatenate list of potential articles by looping through potential articles for each word in query
    for word in queryTerms:
        if word in invertedIndices: #so if someone types in nonsensical query term that's not in invertedIndices, still won't break!
            someArticles = invertedIndices[word]
            potentialArticles = potentialArticles + someArticles
            
    #convert to set then back to list so there are no repeat articles
    potentialArticles = list(set(potentialArticles))
    return potentialArticles

### BM25

Input: daftar artikel, dictionary yang berisi semua dokumen, bobot untuk judul, bobot untuk abstrak, dan query
Output: daftar artikel yang telah diurutkan

Ini adalah metode utama untuk pengambilan informasi yang mengimplementasikan algoritma Okapi BM25.

In [39]:
def bm25(articles, df_dic, title_w, abstract_w, query):
    corpus_title = []
    corpus_abstract = []
    
    for article in articles:
        arr = df_dic.get(article)
        #title
        if type(arr[0]) != float:
            preprocessedTitle = preprocess(arr[0])
            corpus_title.append(preprocessedTitle)
        else:
            corpus_title.append(" ")
        
        #abstract
        if type(arr[1]) != float:
            preprocessedAbst = preprocess(arr[1])
            corpus_abstract.append(preprocessedAbst)
        else:
            corpus_abstract.append(" ")
            
    query = preprocess(query)
    
    tokenized_query = query.split(" ")
    
    tokenized_corpus_title = [doc.split(" ") for doc in corpus_title]
    tokenized_corpus_abstract = [doc.split(" ") for doc in corpus_abstract]
    
    #running bm25 on titles
    bm25_title = BM25Okapi(tokenized_corpus_title)
    doc_scores_titles = bm25_title.get_scores(tokenized_query)
    #weighting array
    doc_scores_titles = np.array(doc_scores_titles)
    doc_scores_titles = doc_scores_titles**title_w
    
    #running bm25 on abstracts
    bm25_abstract = BM25Okapi(tokenized_corpus_abstract)
    doc_scores_abstracts = bm25_abstract.get_scores(tokenized_query)
    #weighting
    doc_scores_abstracts = np.array(doc_scores_abstracts)
    doc_scores_abstracts = doc_scores_abstracts ** abstract_w
    
    #summing up the two different scores
    doc_scores = np.add(doc_scores_abstracts,doc_scores_titles)
    
    #creating a dictionary with the scores
    score_dict = dict(zip(articles, doc_scores))
    
    #creating list of ranked documents high to low
    doc_ranking = sorted(score_dict, key=score_dict.get, reverse = True)
    
    #get top 100
    doc_ranking = doc_ranking[0:100]
    
    for i in range(len(doc_ranking)):
        dic_entry = df_dic.get(doc_ranking[i])
        doc_ranking[i] = dic_entry[0]
    
    return doc_ranking

retrieval step

In [40]:
def retrieve(queries):
    #performing information retrieval
    df_without_keywords = pickle.load(open("full_data_processed_FINAL.p", "rb"))
    df_dic = df_without_keywords.set_index('paper_id').T.to_dict('list')
    results = []
    for q in queries:
        articles = getPotentialArticleSubset(q)
        result = bm25(articles,df_dic,1,2,q)
        results.append(result)

    #Output results
    for query in range(len(results)):
        for rank in range(len(results[query])):
            print(str(query+1)+'\t'+str(rank+1)+'\t'+str(results[query][rank]))

Menggabungkan semuanya :  
Ini akan memberikan kita 100 artikel teratas untuk setiap query yang diurutkan berdasarkan peringkat BM25.

In [41]:
getData()
extract()
organize()
q = ['coronavirus origin',
'coronavirus response to weather changes',
'coronavirus immunity']
retrieve(q)

Directory already exists
Downloaded file 1/4


C:\Users\Tuhu Pangestu\AppData\Local\Temp\ipykernel_11496\2087884789.py:16: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall('./data')


Extracted file 1/4
Downloaded file 2/4
Extracted file 2/4
Downloaded file 3/4
Extracted file 3/4
Downloaded file 4/4
Extracted file 4/4


0it [00:00, ?it/s]
100%|██████████| 2350/2350 [02:22<00:00, 16.47it/s]


1	1	Insights into the cross-species evolution of 2019 novel coronavirus
1	2	Bat origin of human coronaviruses
1	3	
1	4	Origin, Evolution, and Genotyping of Emergent Porcine Epidemic Diarrhea Virus Strains in the United States
1	5	The battle against SARS and MERS coronaviruses: Reservoirs and Animal Models
1	6	Newly emerged porcine enteric alphacoronavirus in southern China: Identification, origin and evolutionary history analysis
1	7	Molecular Evolution of Human Coronavirus Genomes
1	8	MOTS CLÉS
1	9	Bat origin of a new human coronavirus: there and back again
1	10	A review of studies on animal reservoirs of the SARS coronavirus
1	11	SARS-Coronavirus ancestor's foot-prints in South-East Asian bat colonies and the refuge theory
1	12	Testing the hypothesis of a recombinant origin of the SARS-associated coronavirus
1	13	ScienceDirect Focus on Middle East respiratory syndrome coronavirus (MERS-CoV)
1	14	
1	15	Evidence of recombination in coronaviruses implicating pangolin origins of nCoV- 20

References  
https://www.kaggle.com/code/basharzaidat/covid-19-tf-idf-bm25-information-retrieval-system#ORGANIZE